# Checking Transcript Claims

- Uses Azure OpenAI (gpt-4o) with Bing grounding to verify transcript-derived claims  
- Input: Transcript_Claims_FIXED.csv with Video_ID and Claim  
- Prompts model to:
  - Split paragraphs into sub-claims
  - Return for each sub-claim:  
    - Result (1, 0, -1, -2)  
    - Short explanation  
    - 1–3 sources  
- Sends 1 claim per request (batch size = 1)    
- Used to assess the factuality of claims extracted from transcripts


### Importing Libraries

In [ ]:
import pandas as pd
import time
import uuid
import requests
from tqdm import tqdm
import os
import re
import csv

### Cleaning and Expanding Transcript-Level Claims  


In [68]:
# Loading original claims file
df = pd.read_csv("Extracted_Transcript_Claims_FIXED.csv")

# List to collect cleaned claims
expanded_rows = []

for _, row in df.iterrows():
    video_id = row["Video_ID"]
    claims_raw = str(row["Extracted_Claims"]).strip("[]“”‘’\"' ")

    # Split only on commas between quotes
    split_claims = re.split(r"""(?<=['"])\s*,\s*(?=['"])""", claims_raw)

    for claim in split_claims:
        cleaned = claim.strip(" '\"")
        if cleaned:
            expanded_rows.append({"Video_ID": video_id, "Claim": cleaned})

# Create and save final DataFrame
pd.DataFrame(expanded_rows).to_csv("Transcript_Claims_FIXED.csv", index=False)
print("Done. Saved to 'Transcript_Claims_FIXED.csv'")


Done. Saved to 'Transcript_Claims_FIXED.csv'


### Fact-Checking Transcript Claims Using Azure OpenAI with Bing Grounding  


In [ ]:
# --- Azure OpenAI Configuration ---
API_KEY = "***********"
ENDPOINT = "https://***************.openai.azure.com/"
DEPLOYMENT_NAME = "gpt-4o"
API_VERSION = "2024-02-15-preview"

HEADERS = {
    "Content-Type": "application/json",
    "api-key": API_KEY
}

# File Paths
INPUT_CSV = "Transcript_Claims_FIXED.csv"
OUTPUT_CSV = "Transcript_Claims_Results.csv"
CLAIM_BATCH_SIZE = 1

# Load and Filter Data
df = pd.read_csv(INPUT_CSV)
df = df.dropna(subset=["Claim"]).reset_index(drop=True)

# Batch utility
def create_batched_claims(df, batch_size):
    for i in range(0, len(df), batch_size):
        yield df.iloc[i:i + batch_size]

# Prompt Builder
def build_prompt(claims):
    system_msg = (
        "You are a fact-checking assistant. You will be given 1 claim paragraph at a time.\n"
        "Split any paragraph into sub-claims if needed.\n"
        "For each sub-claim, return:\n\n"
        "Sub-Claim N: [text]\n"
        "Result: [1 / 0 / -1 / -2]\n"
        "Explanation: [short paragraph]\n"
        "Sources: [source 1, source 2, source 3]\n\n"
        "Format all sub-claims in one block."
    )

    messages = [{"role": "system", "content": system_msg}]
    for _, row in claims.iterrows():
        messages.append({
            "role": "user",
            "content": f"Video ID: {row['Video_ID']}\nClaim Paragraph: {row['Claim']}"
        })
    return messages

# Save URL
url = f"{ENDPOINT}openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"


In [128]:
# Main Loop
for batch in tqdm(create_batched_claims(df, CLAIM_BATCH_SIZE), total=(len(df) // CLAIM_BATCH_SIZE) + 1):
    messages = build_prompt(batch)

    body = {
        "messages": messages,
        "temperature": 0.2,
        "max_tokens": 1500,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }

    try:
        response = requests.post(url, headers=HEADERS, json=body)

        if response.status_code == 200:
            full_reply = response.json()["choices"][0]["message"]["content"]
            video_id = batch.iloc[0]["Video_ID"]
            claim_text = batch.iloc[0]["Claim"]

            combined_record = {
                "Video_ID": video_id,
                "Claim_Text": claim_text,
                "Claim_Response": full_reply.strip()
            }

            pd.DataFrame([combined_record]).to_csv(
                OUTPUT_CSV,
                mode='a',
                index=False,
                header=not os.path.exists(OUTPUT_CSV),
                quoting=csv.QUOTE_ALL
            )
        else:
            print("Error:", response.status_code)
            print(response.text)

    except Exception as e:
        print("Exception:", str(e))

    time.sleep(1)

print("Processing complete. Output saved to:", OUTPUT_CSV)


  1%|▎                                          | 7/942 [00:21<47:45,  3.06s/it]

Error: 400
{"error":{"message":"The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766","type":null,"param":"prompt","code":"content_filter","status":400,"innererror":{"code":"ResponsibleAIPolicyViolation","content_filter_result":{"hate":{"filtered":false,"severity":"safe"},"jailbreak":{"filtered":false,"detected":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":true,"severity":"medium"}}}}}


  1%|▌                                         | 12/942 [00:42<57:41,  3.72s/it]

Error: 400
{"error":{"message":"The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766","type":null,"param":"prompt","code":"content_filter","status":400,"innererror":{"code":"ResponsibleAIPolicyViolation","content_filter_result":{"hate":{"filtered":false,"severity":"safe"},"jailbreak":{"filtered":false,"detected":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":true,"severity":"medium"}}}}}


  1%|▌                                         | 13/942 [00:45<55:08,  3.56s/it]

...

Error: 400
{"error":{"message":"The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766","type":null,"param":"prompt","code":"content_filter","status":400,"innererror":{"code":"ResponsibleAIPolicyViolation","content_filter_result":{"hate":{"filtered":false,"severity":"low"},"jailbreak":{"filtered":false,"detected":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":true,"severity":"medium"}}}}}


 97%|███████████████████████████████████████▊ | 915/942 [53:16<01:41,  3.76s/it]

Error: 400
{"error":{"message":"The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766","type":null,"param":"prompt","code":"content_filter","status":400,"innererror":{"code":"ResponsibleAIPolicyViolation","content_filter_result":{"hate":{"filtered":false,"severity":"safe"},"jailbreak":{"filtered":false,"detected":false},"self_harm":{"filtered":false,"severity":"safe"},"sexual":{"filtered":false,"severity":"safe"},"violence":{"filtered":true,"severity":"medium"}}}}}


100%|████████████████████████████████████████▉| 941/942 [54:43<00:03,  3.49s/it]

Processing complete. Output saved to: Transcript_Claims_Results.csv


### Extracting and Structuring Sub-Claims from GPT Fact-Check Responses


In [10]:
# Load responses
df = pd.read_csv("Transcript_Claims_Results.csv")
rows = []

# Parse each GPT response
for _, row in df.iterrows():
    for block in re.split(r"Sub-Claim\s+\d+:", str(row["Claim_Response"])):
        block = block.strip()
        if not block:
            continue

        rows.append({
            "Video_ID": row["Video_ID"],
            "Original_Claim_Text": row["Claim_Text"],
            "Sub_Claim": block.split("Result:")[0].strip() if "Result:" in block else "",
            "Result": re.search(r"Result:\s*\[?(-?[012])\]?", block).group(1).strip() if re.search(r"Result:\s*\[?(-?[012])\]?", block) else "",
            "Explanation": re.search(r"Explanation:\s*(.*?)(?:Sources:|$)", block, re.DOTALL).group(1).strip() if re.search(r"Explanation:\s*(.*?)(?:Sources:|$)", block, re.DOTALL) else "",
            "Sources": re.search(r"Sources:\s*(.*)", block, re.DOTALL).group(1).strip() if re.search(r"Sources:\s*(.*)", block, re.DOTALL) else ""
        })

# Save to CSV
pd.DataFrame(rows).to_csv("Transcript_Claims_SubClaims_Expanded.csv", index=False)
print("Saved to Transcript_Claims_SubClaims_Expanded.csv")


Saved to Transcript_Claims_SubClaims_Expanded.csv
